In [8]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair
json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
events = {}

evt: LogResult
for index, evt in enumerate(ReadEvents().apply(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
)):
    
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['event_type'] = evt["event"]
    event['event_details'] = {}
    event['event_details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['event_details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['event_details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['event_details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[index] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,278,965 tx:0x48cc0193c7b1eaeea89af79d89860675eb0ea8133350ec396f852923992abdbe
Swap at block:60,278,965 tx:0xceff08d31298bc30b21c563fbc8e5640dae8017563840aafce6a42d89666ab16
Swap at block:60,278,968 tx:0x3ff555ad662b7e412057b5fa2271c570944c3a8b539b9a51ba30c2eb385c6869
Swap at block:60,278,968 tx:0x5e0cbf00a761694adc84040835ac289b6bd0f595b9b565b2077c920816c633ee
Swap at block:60,278,970 tx:0xf7b6d3aeb78f3506c5c18fd3c726201ed636b7efc833cfab2c449ebb9a70c816
Swap at block:60,278,970 tx:0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd1660f4ca80b31b21b852da9
Swap at block:60,278,970 tx:0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd1660f4ca80b31b21b852da9
Swap at block:60,278,970 tx:0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd1660f4ca80b31b21b852da9
Swap at block:60,278,970 tx:0xb85a2cce9d33bc51eccb6a4aff16533e5be8b9ba82636fd4ecd4ffd573311dc0
Swap at block:60,278,970 tx:0xb85a2cce9d33bc51eccb6a4aff16533e5be8b9ba82636fd4ecd4ffd573311dc0
Swap at block:60,278,970 tx:0x03a68863b7d997f3d191

In [9]:
pd.DataFrame.from_dict(events, orient='index') 

,chain,platform,contract,address,tx_hash,blk_num,timestamp,event_type,event_details
0,polygon,sushi,UniswapV2Pair,0x869659a65d2d5860c146b41254f10c80b50125ba,0x48cc0193c7b1eaeea89af79d89860675eb0ea8133350...,60278965,1722985682,<class 'web3._utils.datatypes.Swap'>,{'token0': '0x93300E631Ec66E918811785598476F73...
1,polygon,sushi,UniswapV2Pair,0xdef834ae4a1198bfec84544fb374ec7f1314a7c5,0xceff08d31298bc30b21c563fbc8e5640dae801756384...,60278965,1722985682,<class 'web3._utils.datatypes.Swap'>,{'token0': '0x1111111254EEB25477B68fb85Ed929f7...
2,polygon,sushi,UniswapV2Pair,0x9eb792c780d3e3714f245c596c8c8993bf6fc605,0x3ff555ad662b7e412057b5fa2271c570944c3a8b539b...,60278968,1722985688,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xf081470f5C6FBCCF48cC4e5B82Dd9264...
3,polygon,sushi,UniswapV2Pair,0xe27855cce2ddc3a2e44f1ad3cdc3ec6ce4903bba,0x5e0cbf00a761694adc84040835ac289b6bd0f595b9b5...,60278968,1722985688,<class 'web3._utils.datatypes.Swap'>,{'token0': '0x37af74b8096A6fd85bc4A36653a60B8D...
4,polygon,sushi,UniswapV2Pair,0xfcacacc6c92d93f76d34e51d21d28227b3c30563,0xf7b6d3aeb78f3506c5c18fd3c726201ed636b7efc833...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0x93300E631Ec66E918811785598476F73...
5,polygon,sushi,UniswapV2Pair,0x9803c7ae526049210a1725f7487af26fe2c24614,0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd166...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xE37e799D5077682FA0a244D46E5649F7...
6,polygon,sushi,UniswapV2Pair,0x5c981300a2fd35aed8427b2d6d004e0ab11602b0,0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd166...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xE37e799D5077682FA0a244D46E5649F7...
7,polygon,sushi,UniswapV2Pair,0x7805b64e2d99412d3b8f10dfe8fc55217c5cc954,0xfc485ab9b39b9a5d98b7a429799eff2145bbfe1dd166...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xE37e799D5077682FA0a244D46E5649F7...
8,polygon,sushi,UniswapV2Pair,0x1e67124681b402064cd0abe8ed1b5c79d2e02f64,0xb85a2cce9d33bc51eccb6a4aff16533e5be8b9ba8263...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xf5F7cb5DA2E4d905fC4fE0a3A48ee6b2...
9,polygon,sushi,UniswapV2Pair,0x5c981300a2fd35aed8427b2d6d004e0ab11602b0,0xb85a2cce9d33bc51eccb6a4aff16533e5be8b9ba8263...,60278970,1722985692,<class 'web3._utils.datatypes.Swap'>,{'token0': '0xf5F7cb5DA2E4d905fC4fE0a3A48ee6b2...


In [7]:
events

{0: {'chain': 'polygon',
  'platform': 'sushi',
  'contract': 'UniswapV2Pair',
  'address': '0x869659a65d2d5860c146b41254f10c80b50125ba',
  'tx_hash': '0x48cc0193c7b1eaeea89af79d89860675eb0ea8133350ec396f852923992abdbe',
  'blk_num': 60278965,
  'timestamp': 1722985682,
  'event_type': web3._utils.datatypes.Swap,
  'event_details': {'token0': '0x93300E631Ec66E918811785598476F738DA7d5EB',
   'token1': '0x93300E631Ec66E918811785598476F738DA7d5EB',
   'amount0In': 0,
   'amount1Out': 0}},
 1: {'chain': 'polygon',
  'platform': 'sushi',
  'contract': 'UniswapV2Pair',
  'address': '0xdef834ae4a1198bfec84544fb374ec7f1314a7c5',
  'tx_hash': '0xceff08d31298bc30b21c563fbc8e5640dae8017563840aafce6a42d89666ab16',
  'blk_num': 60278965,
  'timestamp': 1722985682,
  'event_type': web3._utils.datatypes.Swap,
  'event_details': {'token0': '0x1111111254EEB25477B68fb85Ed929f73A960582',
   'token1': '0x1111111254EEB25477B68fb85Ed929f73A960582',
   'amount0In': 0,
   'amount1Out': 0}},
 2: {'chain': 'pol